In [1]:
import glob
import os
import warnings

import cv2  # 画像認識するのはcv2
import keras
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import plotly
import plotly.graph_objs as go
import plotly.offline as offline
import seaborn as sns
import tensorflow as tf
from keras import backend as K
from keras import layers
from keras.backend import set_session
from keras.callbacks import CSVLogger
from keras.layers import *
from keras.models import Model
from keras.utils.np_utils import to_categorical
from plotly.offline import init_notebook_mode
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from tqdm import tqdm

from functions import *

warnings.simplefilter('ignore')

Using TensorFlow backend.
/home/tsutsumi/.conda/envs/TFGPU223_20201217/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tsutsumi/.conda/envs/TFGPU223_20201217/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tsutsumi/.conda/envs/TFGPU223_20201217/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.in

In [2]:
X, y = load_data()
filelist = pd.read_csv("/home/tsutsumi/BONE/MORPHO-VAE/CODE/new_Mandible_check.csv", encoding="SHIFT-JIS")
arch_list = pd.read_csv('architecture_list.csv')

"""
    Functions
"""
def vae_loss(inputs, z_decoded):
        x = K.flatten(inputs)
        z_decoded = K.flatten(z_decoded)
        xent_loss = keras.metrics.binary_crossentropy(x, z_decoded)
        kl_loss = -5e-4 * K.mean(
            1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return K.mean(xent_loss + kl_loss)


In [3]:
alpha = 0.1 ## Morpho-VAE
latent_dim = 3
date = '20220209'
val_seed = 42
for test_seed,num in zip([230],[7]):
    #K.clear_session()
    config = tf.ConfigProto(
        gpu_options=tf.GPUOptions(
            visible_device_list="2", # specify GPU number
            allow_growth=True
        )
    )
    set_session(tf.Session(config=config))
    X_train, y_train,Y_train, X_test, y_test , Y_test, group_train, group_test =  make_train_test(X,y,seed = test_seed)
    ## make val data
    X_train_2, y_train_2,Y_train_2, X_val, y_val ,Y_val, val_seed, group_train_2, group_val = \
    make_val_train(X_train,y_train,group_train,seed = val_seed)
    
    checkpoint = keras.callbacks.ModelCheckpoint("/home/tsutsumi/BONE/MORPHO-VAE/CODE/WEIGHTS/alpha_{}.h5".format(test_seed),
                         monitor="val_loss",
                         mode="min",
                         save_best_only = True,
                         verbose=0)

    earlystop = keras.callbacks.EarlyStopping(monitor = 'val_loss', 
                              min_delta = 0, 
                              patience = 10,
                              verbose = 0)

    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss',
                                  factor = 0.5,
                                  patience = 10,
                                  verbose = 0)

    csv_logger = CSVLogger('/home/tsutsumi/BONE/MORPHO-VAE/CODE/HISTORY/history_{0}_alpha{1}_testseed{2}_valseed{3}_2.csv'.format(date,alpha, test_seed,val_seed), append=True, separator=',')

    callbacks = [checkpoint, csv_logger] #[earlystop, checkpoint, reduce_lr,csv_logger]

    
    model, z_mean, z_log_var, encoder, decoder, Classifier = create_model(arch_list.iloc[num,0], arch_list.iloc[num,1], list((arch_list.iloc[num,2:7]).astype(int)),3)

    optimizer = arch_list.iloc[num,8]
    model.compile(optimizer=optimizer, 
              loss = ['categorical_crossentropy',vae_loss], 
              loss_weights = [alpha, 1 - alpha],
              metrics=['accuracy'])
    
    model.fit(X_train_2,
                      [Y_train_2, X_train_2],
                      epochs=100,
                      batch_size=10,
                      validation_data=(X_val,[Y_val,X_val]),
                      callbacks = callbacks,
                     verbose = 2)

    #os.makedirs('/home/tsutsumi/BONE/MORPHO-VAE/CODE/WEIGHTS/Publish/',exist_ok=True)
    #os.makedirs('/home/tsutsumi/BONE/MORPHO-VAE/CODE/WEIGHTS/Publish/{}/{}'.format(date,test_seed), exist_ok=True)


    #encoder.save_weights('/home/tsutsumi/BONE/MORPHO-VAE/CODE/WEIGHTS/Publish/{0}/{2}/encoder_{0}_alpha{1}_testseed{2}_valseed{3}.h5'.format(date,alpha, test_seed,val_seed))
    #decoder.save_weights('/home/tsutsumi/BONE/MORPHO-VAE/CODE/WEIGHTS/Publish/{0}/{2}/decoder_{0}_alpha{1}_testseed{2}_valseed{3}.h5'.format(date,alpha, test_seed,val_seed))
    #Classifier.save_weights('/home/tsutsumi/BONE/MORPHO-VAE/CODE/WEIGHTS/Publish/{0}/{2}/Classifier_{0}_alpha{1}_testseed{2}_valseed{3}.h5'.format(date,alpha, test_seed,val_seed))
    #model.save_weights('/home/tsutsumi/BONE/MORPHO-VAE/CODE/WEIGHTS/Publish/{0}/{2}/model_{0}_alpha{1}_testseed{2}_valseed{3}.h5'.format(date,alpha, test_seed,val_seed))



Train size :(195, 128, 128, 3), Test size:(99, 128, 128, 3)
Train2 size :(145, 128, 128, 3), val size:(50, 128, 128, 3)
Train on 145 samples, validate on 50 samples
Epoch 1/100
 - 12s - loss: 0.3998 - Family_Classifier_loss: 1.9021 - model_2_loss: 0.2328 - Family_Classifier_acc: 0.4483 - model_2_acc: 0.3342 - val_loss: 0.2660 - val_Family_Classifier_loss: 1.7033 - val_model_2_loss: 0.1063 - val_Family_Classifier_acc: 0.6000 - val_model_2_acc: 0.5891
Epoch 2/100
 - 1s - loss: 0.2431 - Family_Classifier_loss: 1.5306 - model_2_loss: 0.1000 - Family_Classifier_acc: 0.5793 - model_2_acc: 0.3100 - val_loss: 0.2441 - val_Family_Classifier_loss: 1.5894 - val_model_2_loss: 0.0946 - val_Family_Classifier_acc: 0.6000 - val_model_2_acc: 0.7981
Epoch 3/100
 - 1s - loss: 0.2265 - Family_Classifier_loss: 1.4251 - model_2_loss: 0.0933 - Family_Classifier_acc: 0.6138 - model_2_acc: 0.5581 - val_loss: 0.2670 - val_Family_Classifier_loss: 1.5301 - val_model_2_loss: 0.1266 - val_Family_Classifier_acc: 0.6